This code creates files with interaction probabilies between prey and predator for every parameter configuration.

# CA

In [ ]:
! pip install pygame -q

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
from google.colab import output
import time
import os, sys
import random
import json
import math

import pygame
import numpy as np
import matplotlib.pyplot as plt

pygame 2.5.2 (SDL 2.28.2, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
os.environ["SDL_VIDEODRIVER"] = "dummy"

In [ ]:
class State:
    def __init__(self, field_size, prey_population, predator_population, prey_death_rate, predator_death_rate, prey_birth_rate, predator_birth_rate):
        self.field_size = field_size
        self.prey_population = prey_population
        self.predator_population = predator_population
        self.field = self.init_field(field_size, prey_population, predator_population)
        self._prey_death_rate = prey_death_rate
        self._predator_death_rate = predator_death_rate
        self._prey_birth_rate = prey_birth_rate
        self._predator_birth_rate = predator_birth_rate

    def init_field(self, field_size, prey_population, predator_population):
        field = np.zeros((field_size, field_size))

        prey_location = np.random.choice(field_size*field_size, prey_population, replace=False)
        predator_location = np.random.choice(np.setdiff1d(np.arange(field_size*field_size), prey_location), predator_population, replace=False)

        prey_index = np.unravel_index(prey_location, (field_size, field_size))
        predator_index = np.unravel_index(predator_location, (field_size, field_size))

        field[prey_index] = 1
        field[predator_index] = 2

        return field

    def take_step(self):
        interactions = 0
        newField = np.array(self.field)

        for (x, y), element in np.ndenumerate(self.field):
            if element == 1:
                r = np.random.uniform(0, 1)
                predators_in_neighboorhood = self.count_target_moore(x,y,2)
                if predators_in_neighboorhood != 0:
                    interactions += 1
                if not (r < (1-self._prey_death_rate)**predators_in_neighboorhood):
                    r = np.random.uniform(0, 1)
                    if r < self._predator_birth_rate:
                        newField[x,y] = 2
                        self.predator_population += 1
                        self.prey_population -= 1
            if element == 2:
                r = np.random.uniform(0, 1)
                if r < self._predator_death_rate:
                    newField[x,y] = 0
                    self.predator_population -= 1
            if element == 0:
                predators_in_neighboorhood = self.count_target_moore(x,y,2)
                preys_in_neighboorhood = self.count_target_moore(x,y,1)
                if preys_in_neighboorhood > 0 and predators_in_neighboorhood == 0:
                    r = np.random.uniform(0, 1)
                    if r < self._prey_birth_rate**preys_in_neighboorhood:
                        newField[x,y] = 1
                        self.prey_population += 1
        self.field = newField
        self.prey_population = int(np.sum(self.field == 1))
        self.predator_population = int (np.sum(self.field == 2))
        return interactions

    def move_random(self):
        newField = np.zeros((self.field_size, self.field_size))

        for (x, y), element in np.ndenumerate(self.field):
            if element == 1 or element == 2:
                empty_neighbours = self.find_neighbours_moore(newField, x, y, 0)
                if not empty_neighbours:
                    newField[x,y] = element
                else:
                    pos_change = random.choice(empty_neighbours)
                    newField[pos_change[0], pos_change[1]] = element
        self.field = newField

    def move_with_purpose(self):
        newField = np.zeros((self.field_size, self.field_size))

        for (x, y), element in np.ndenumerate(self.field):
            if element == 1:
                empty_neighbours = self.find_neighbours_moore(newField, x, y, 0)
                predators = self.find_neighbours_moore(self.field, x, y, 2)
                if not empty_neighbours:
                    newField[x,y] = element
                else:
                    scores = []
                    if not predators:
                        pos_change = random.choice(empty_neighbours)
                    else:
                        for xn, yn in empty_neighbours:
                            #calculate which neighbour is furthest from the predators
                            score = 0
                            for px, py in predators:
                                score += math.dist([xn, yn], [px, py])
                            scores.append(score)
                        min_score = max(scores) #check if unique!!!!!!!!!!!!
                        index = scores.index(min_score)
                        pos_change = empty_neighbours[index]
                    newField[pos_change[0], pos_change[1]] = element

            if element == 2:
                empty_neighbours = self.find_neighbours_moore(newField, x, y, 0)
                prey = self.find_neighbours_moore(self.field, x, y, 1)
                if not empty_neighbours:
                    newField[x,y] = element
                else:
                    scores = []
                    if not prey:
                        pos_change = random.choice(empty_neighbours)
                    else:
                        for xn, yn in empty_neighbours:
                            #calculate which neighbour is furthest from the predators
                            score = 0
                            for px, py in prey:
                                score += math.dist([xn, yn], [px, py])
                            scores.append(score)
                        min_score = min(scores)
                        index = scores.index(min_score)
                        pos_change = empty_neighbours[index]
                    newField[pos_change[0], pos_change[1]] = element

        self.field = newField

    def find_neighbours_moore(self, field, x, y, target):
        offsets = [(-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1),  (1, 0),  (1, 1)]
        neighbours = []

        for dr, dc in offsets:
            r_neighbor = x + dr
            c_neighbor = y + dc
            if r_neighbor >= self.field_size:
                r_neighbor = 0
            if r_neighbor < 0:
                r_neighbor = self.field_size - 1
            if c_neighbor >= self.field_size:
                c_neighbor = 0
            if c_neighbor < 0:
                c_neighbor = self.field_size - 1
            if field[r_neighbor, c_neighbor] == target:
                neighbours.append((r_neighbor, c_neighbor))
        return neighbours


    def count_target_moore(self, x, y, target):
        offsets = [(-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1),  (1, 0),  (1, 1)]
        count = 0

        for dr, dc in offsets:
            r_neighbor = x + dr
            c_neighbor = y + dc
            if r_neighbor >= self.field_size:
                r_neighbor = 0
            if r_neighbor < 0:
                r_neighbor = self.field_size - 1
            if c_neighbor >= self.field_size:
                c_neighbor = 0
            if c_neighbor < 0:
                c_neighbor = self.field_size - 1
            if self.field[r_neighbor, c_neighbor] == target:
                count += 1
        return count

In [ ]:
BLACK = (0, 0, 0)
WHITE = (200, 200, 200)
RED = (255, 0, 0)
GREEN = (0, 255, 0)

#FIELD_SIZE = 10
FIELD_SIZE = 25
#FIELD_SIZE = 35
FIELD_HEIGHT = FIELD_SIZE
FIELD_WIDTH = FIELD_SIZE
PREY_POPULATION = 100
PREDATOR_POPULATION = 66
#PREY_POPULATION = 4
#PREDATOR_POPULATION = 2

# VISUALIZATION PARAMETERS
CELL_SIZE = 10 #Set the size of the grid block
WINDOW_HEIGHT = FIELD_HEIGHT * CELL_SIZE
WINDOW_WIDTH = FIELD_WIDTH * CELL_SIZE

def main(mode: str=None, prey_death_rate=0, predator_birth_rate=0, movement=None, **kwargs):
    global SCREEN, CLOCK
    prey_history = []
    predator_history = []
    interaction_history = []
    pygame.init()
    SCREEN = pygame.display.set_mode((WINDOW_WIDTH, WINDOW_HEIGHT))
    CLOCK = pygame.time.Clock()
    SCREEN.fill(BLACK)

    state = State(FIELD_SIZE,
                  PREY_POPULATION,
                  PREDATOR_POPULATION,
                  prey_death_rate=prey_death_rate,
                  predator_death_rate=0.3,
                  prey_birth_rate=0.7,
                  predator_birth_rate=predator_birth_rate)

    prey_history.append(state.prey_population)
    predator_history.append(state.predator_population)
    if mode == 'ONE-STEP':
        simulate_one_step(state, prey_history, predator_history, movement, interaction_history)
    elif mode == 'WHILE':
        while True:
            simulate_one_step(state, prey_history, predator_history, movement, interaction_history)
    elif mode == 'FOR':
        step_n = kwargs.get('step_n', 10)
        for i in range(step_n):
            simulate_one_step(state, prey_history, predator_history, movement, interaction_history)

    return prey_history, predator_history, interaction_history

def simulate_one_step(state, prey_history, predator_history, movement, interaction_history):
    # SOMEWHERE HERE SHOULD BE UPDATE STATE
    if movement == 'random':
        state.move_random()
    if movement == 'hunt':
        state.move_with_purpose()
    if predator_history[-1] == 0 and prey_history[-1] == 0:
        interaction_history.append((state.take_step()))
    elif predator_history[-1] == 0:
        interaction_history.append((state.take_step())/(prey_history[-1]))
    elif prey_history[-1] == 0:
        interaction_history.append((state.take_step())/(predator_history[-1]))
    else:
        interaction_history.append((state.take_step())/(prey_history[-1] * predator_history[-1]))
    prey_history.append(state.prey_population)
    predator_history.append(state.predator_population)
    state.field = state.init_field(state.field_size, state.prey_population, state.predator_population)


In [ ]:
def cross_correlation_analysis(ts1, ts2):
    # Calculate cross-correlation
    cross_corr = np.correlate(ts1, ts2, mode='full')

    # Find lag with maximum correlation
    max_corr_index = np.argmax(cross_corr)
    lag = max_corr_index - len(ts1) + 1  # Lag is relative to ts1

    # Calculate correlation coefficient
    correlation_coefficient = np.corrcoef(ts1, np.roll(ts2, lag))[0, 1]

    return lag, correlation_coefficient

# Get Interaction Parameters

In [ ]:
for i in range(1,6):
    json_data = {}
    count = 0
    for prey_death_rate in np.arange(0, 1.05, 0.05):
        prey_death_rate = round(prey_death_rate, 2)
        for predator_birth_rate in np.arange(0, 1.05, 0.05):
            count = count + 1
            predator_birth_rate = round(predator_birth_rate, 2)
            prey_history, predator_history, interaction_history = main('FOR', prey_death_rate=prey_death_rate, predator_birth_rate=predator_birth_rate, movement=None, step_n=200)
            lag, correlation = cross_correlation_analysis(prey_history, predator_history)
            current_data = {'prey_death_rate': prey_death_rate,
                            'predator_birth_rate': predator_birth_rate,
                            'interaction_history': interaction_history,
                            'interaction_mean': np.mean(interaction_history),
                            }
            json_data[str(count)] = current_data
            with open(f"/content/drive/MyDrive/NaCo_finalProject/experiments/run_reshuffel_{i}.json", "w") as json_file:
                json.dump(json_data, json_file, indent=4)
        print(f'-- FINISHED Prey_Death_rate {prey_death_rate} --')
    print(f'-- FINISHED Iteration {i} --')

-- FINISHED Prey_Death_rate 0.0 --
-- FINISHED Prey_Death_rate 0.05 --
-- FINISHED Prey_Death_rate 0.1 --
-- FINISHED Prey_Death_rate 0.15 --
-- FINISHED Prey_Death_rate 0.2 --
-- FINISHED Prey_Death_rate 0.25 --
-- FINISHED Prey_Death_rate 0.3 --
-- FINISHED Prey_Death_rate 0.35 --
-- FINISHED Prey_Death_rate 0.4 --
-- FINISHED Prey_Death_rate 0.45 --
-- FINISHED Prey_Death_rate 0.5 --
-- FINISHED Prey_Death_rate 0.55 --
-- FINISHED Prey_Death_rate 0.6 --
-- FINISHED Prey_Death_rate 0.65 --
-- FINISHED Prey_Death_rate 0.7 --
-- FINISHED Prey_Death_rate 0.75 --
-- FINISHED Prey_Death_rate 0.8 --
-- FINISHED Prey_Death_rate 0.85 --
-- FINISHED Prey_Death_rate 0.9 --
-- FINISHED Prey_Death_rate 0.95 --
-- FINISHED Prey_Death_rate 1.0 --
-- FINISHED Iteration 1 --
-- FINISHED Prey_Death_rate 0.0 --
-- FINISHED Prey_Death_rate 0.05 --
-- FINISHED Prey_Death_rate 0.1 --
-- FINISHED Prey_Death_rate 0.15 --
-- FINISHED Prey_Death_rate 0.2 --
-- FINISHED Prey_Death_rate 0.25 --
-- FINISHED Pre